In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

# ATMCS

In [16]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 62 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 61 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 61 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 61 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 61 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 62 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 61 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 61 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) f

Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

# ATPtg

In [15]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 54 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 52 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 10 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 54 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 54 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 11 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 10 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure)

Done


[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=243., elDegs=80.)

In [6]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [7]:
ra = Angle("14:39:36.20", unit=u.hour)
dec = Angle("-60:50:08.2", unit=u.deg)
target_name="Alf Cen"
radec = ICRS(ra, dec)


In [18]:
ra = Angle("17:03:39", unit=u.hour)
dec = Angle("-22:31:19", unit=u.deg)
target_name="Jupiter"
radec = ICRS(ra, dec)



In [ ]:
await atmcs.cmd_stopTracking.start()

In [13]:
await atptg.cmd_stopTracking.start(timeout=10)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 41 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 41 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 41 messages


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2071115982, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [19]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling 

58749.95408311977
slew...


RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 37 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
ss = await atdome.evt_summaryState.next(flush=False, timeout=5)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
ss = await atdomtraj.evt_summaryState.next(flush=False, timeout=5)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await atdomtraj.cmd_enable.start()

In [20]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'n':
            print("Next point in the grid...")
            return 1
        elif opt == 'p':
            print("Previous point in the grid...")
            return -1
        elif opt == 't':
            print("Stopping test...")
            return 0
        elif opt == '?':
            print("""Options are:
            n - Go to next point in the grid.
            p - Go to previous point in the grid.
            t - Terminate test.
            """)
        else:
            print("Next point in the grid...")
            return 1

In [21]:
dalt_grid = np.arange(60.*60, 60.*60, 60)
daz_grid = np.arange(60.*60, 60.*60, 60)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0.45*60.*60., 
                                     az=-0.35*60*60, 
                                         num=0)

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=0

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=0., 
                                     num=0)

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                         az=grid_az[i]*60*60, 
                                         num=0)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=-100, 
                                     num=0)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., 
                                     az=0, 
                                         num=1)

In [ ]:
60-61

In [ ]:
await asyncio.sleep(5)
off = await atptg.tel_guidingAndOffsets.next(flush=True, timeout=5)

In [ ]:
print(off)